# Imports

In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Time series
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

# ML
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

import warnings
warnings.filterwarnings('ignore')

# Configurations
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Loading data

In [169]:
# Load sales and market data
sales_df = pd.read_csv('../Data/Case2_Sales data.csv', sep=";")
market_df = pd.read_excel('/Users/diogocarvalho/Documents/GitHub/Case-2--Siemens-Sales-Forecast/Data/Case2_Market data.xlsx', header=[0,1,2])

# Preprocessing sales data

> Sales_EUR is object and must be numeric

In [170]:
# fix the type of Sales_EUR column
sales_df['Sales_EUR'] = (
    sales_df['Sales_EUR']
    .replace(',','.', regex=True)        
    .replace(r'[^0-9\.\-]', '', regex=True) 
    .pipe(pd.to_numeric, errors='coerce')
)

In [171]:
sales_df['Sales_EUR'].dtype

dtype('float64')

> DATE is object, and must be converted to datetime

In [172]:
# Convert the DATE column to datetime
sales_df['DATE'] = pd.to_datetime(sales_df['DATE'], format='%d.%m.%Y')

In [173]:
# Aggregate to monthly level
sales_df['YearMonth'] = sales_df['DATE'].dt.to_period('M')
monthly_sales = sales_df.groupby(['YearMonth', 'Mapped_GCK'])['Sales_EUR'].sum().reset_index()

# Convert YearMonth to datetime
monthly_sales['YearMonth'] = monthly_sales['YearMonth'].dt.to_timestamp()

In [174]:
# make DATE index of sales_df
# sales_df = sales_df.set_index('DATE')

In [175]:
sales_df.head()

,DATE,Mapped_GCK,Sales_EUR,YearMonth
0,2018-10-01,#1,0.0,2018-10
1,2018-10-02,#1,0.0,2018-10
2,2018-10-03,#1,0.0,2018-10
3,2018-10-04,#1,0.0,2018-10
4,2018-10-05,#1,0.0,2018-10


> Aggregating sales per product based on month and year

Since we will be doing monthly analysis, it's no use for us to have the data in days.

In [176]:
# Aggregate sales data by month for each product
monthly_sales_data = sales_df.groupby(['YearMonth', 'Mapped_GCK'], as_index=False).agg({
    'Sales_EUR': 'sum'  # Sum sales revenue
})

In [177]:
# Display the first few rows to check the transformation
monthly_sales_data.head(25)

,YearMonth,Mapped_GCK,Sales_EUR
0,2018-10,#1,36098918.79
1,2018-10,#11,1021303.50
2,2018-10,#12,28686.33
3,2018-10,#13,27666.10
4,2018-10,#14,5770.00
5,2018-10,#16,333196.87
6,2018-10,#20,4563.14
7,2018-10,#3,8089465.96
8,2018-10,#36,6474.60
9,2018-10,#4,397760.69


In [178]:
monthly_sales_data.shape

(602, 3)

### Separate products into different datasets

We will predcit procuts individually, so separating them will be useful

In [179]:
# replace the "#" with "Product" in the product names
monthly_sales_data['Mapped_GCK'] = monthly_sales_data['Mapped_GCK'].str.replace('#', 'Product')

In [180]:
# Get the unique products
unique_products = monthly_sales_data['Mapped_GCK' ].unique()
unique_products

array(['Product1', 'Product11', 'Product12', 'Product13', 'Product14',
       'Product16', 'Product20', 'Product3', 'Product36', 'Product4',
       'Product5', 'Product6', 'Product8', 'Product9'], dtype=object)

In [181]:
# Create a dictionary to store the DataFrames
product_dataframes = {product: monthly_sales_data[monthly_sales_data['Mapped_GCK'] == product] for product in unique_products}

In [182]:
# Loop through each product and assign it to a dynamically named variable
for i, product in enumerate(product_dataframes.keys(), start=1):
    globals()[f"product{i}"] = product_dataframes[product]

In [183]:
product1.head()

,YearMonth,Mapped_GCK,Sales_EUR
0,2018-10,Product1,36098918.79
14,2018-11,Product1,5140760.00
28,2018-12,Product1,37889612.12
42,2019-01,Product1,27728148.35
56,2019-02,Product1,34793163.53


In [184]:
# Check the shape of each product DataFrame
for i in range(1, len(product_dataframes) + 1):
    print(f"product{i} shape: {globals()[f'product{i}'].shape}")

product1 shape: (43, 3)
product2 shape: (43, 3)
product3 shape: (43, 3)
product4 shape: (43, 3)
product5 shape: (43, 3)
product6 shape: (43, 3)
product7 shape: (43, 3)
product8 shape: (43, 3)
product9 shape: (43, 3)
product10 shape: (43, 3)
product11 shape: (43, 3)
product12 shape: (43, 3)
product13 shape: (43, 3)
product14 shape: (43, 3)


In [185]:
for i in range(1, 15):
    df = globals()[f'product{i}']
    
    # Convert Period to Timestamp (datetime type)
    df['DATE'] = df['YearMonth'].dt.to_timestamp()
    
    globals()[f'product{i}'] = df

In [186]:
# check if the conversion was successful
# for i in range(1, len(product_dataframes) + 1):
#     print(f"product{i} first 5 rows:{globals()[f'product{i}'].info()}")

# Preprocessing market data

In [187]:
market_df.head()

Unnamed: 0_level_0  \
  Index 2010=100 (if not otherwise noted)   
                                     date   
0                                  2004m2   
1                                  2004m3   
2                                  2004m4   
3                                  2004m5   
4                                  2004m6   

                                     China  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO156   
0                                16.940704   
1                                23.711852   
2                                24.435235   
3                                23.708115   
4                                27.009138   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP156   
0                               16.940704   
1                               23.711852   
2                               24.435235   
3                               23.708115   
4                               27.009138   

                                    France  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO250   
0                               112.091273   
1                               136.327976   
2                               117.791806   
3                               109.002541   
4                               133.785737   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP250   
0                               83.458866   
1                              106.168192   
2                               92.007646   
3                               85.696486   
4                              106.641482   

                                   Germany  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO276   
0                                82.623037   
1                               100.556582   
2                                89.653203   
3                                86.880571   
4                                99.010814   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP276   
0                               79.452532   
1                               97.012918   
2                               84.932358   
3                               82.372794   
4                               95.108740   

                                     Italy  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO380   
0                               124.289603   
1                               143.411662   
2                               129.083828   
3                               135.590391   
4                               136.424935   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP380   
0                               86.560493   
1                              106.344544   
2                               95.579673   
3                              100.087039   
4                              110.889719   

                                     Japan  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO392   
0                               109.334010   
1                               140.884616   
2                               105.853579   
3                               101.864777   
4                               120.332920   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP392   
0                              110.495272   
1                              144.686166   
2                              102.655769   
3                              100.305285   
4                              119.616380   

            

In [188]:
# drop the second header row
market_df.columns = market_df.columns.droplevel(1)

In [189]:
# Join the header levels
market_df.columns = [
    '_'.join([str(level) for level in col_tuple]).strip()
    for col_tuple in market_df.columns
]


In [190]:
# change first column name to "DATE"
market_df.rename(columns={'Unnamed: 0_level_0_date': 'DATE'}, inplace=True)

In [191]:
# Replace the "m" with "-" in DATE column of market_df
market_df['DATE'] = market_df['DATE'].str.replace('m', '-')

In [192]:
# Convert the DATE column to datetime
market_df['DATE'] = pd.to_datetime(market_df['DATE'], infer_datetime_format=True)

In [193]:
# Create the YearMonth column, like done before in the sales_df
market_df['YearMonth'] = market_df['DATE'].dt.to_period('M')

In [194]:
# Convert YearMonth to match sales_df
market_df['YearMonth'] = market_df['YearMonth'].dt.strftime('%Y-%m')

# Merge market data

In [195]:
for i in range(1, 15):
    df = globals()[f'product{i}']
    
    # Ensure both sides have datetime-type DATE columns (just in case)
    df['DATE'] = pd.to_datetime(df['DATE'])
    market_df['DATE'] = pd.to_datetime(market_df['DATE'])

    # Merge market_df into the product sales data
    merged_df = pd.merge(df, market_df, on='DATE', how='left')

    # Optional: sort by date (and product, if exists)
    merged_df = merged_df.sort_values(['DATE']).reset_index(drop=True)
    
    # Save back
    globals()[f'product{i}'] = merged_df

In [196]:
# # Ensure macro data is monthly
# market_df['YearMonth'] = market_df['DATE'].dt.to_period('M').dt.to_timestamp()

# # Merge with sales
# full_df = pd.merge(monthly_sales, market_df, on='YearMonth', how='left')

# # Sort for time series modeling
# full_df = full_df.sort_values(['Mapped_GCK', 'YearMonth']).reset_index(drop=True)

In [197]:
# check for rows where YearMonth_x is not equal to YearMonth_y
for i in range(1, 15):
    df = globals()[f'product{i}']
    print(f"product{i} shape: {df[df['YearMonth_x'] != df['YearMonth_y']].shape}")

product1 shape: (0, 52)
product2 shape: (0, 52)
product3 shape: (0, 52)
product4 shape: (0, 52)
product5 shape: (0, 52)
product6 shape: (0, 52)
product7 shape: (0, 52)
product8 shape: (0, 52)
product9 shape: (0, 52)
product10 shape: (0, 52)
product11 shape: (0, 52)
product12 shape: (0, 52)
product13 shape: (0, 52)
product14 shape: (0, 52)


In [198]:
# drop the YearMonth_y column in all product DataFrames because it's same as YearMonth_x
for i in range(1, 15):
    df = globals()[f'product{i}']
    df.drop(columns='YearMonth_y', inplace=True)
    globals()[f'product{i}'] = df

In [199]:
# change the name of YearMonth_x to YearMonth
for i in range(1, 15):
    df = globals()[f'product{i}']
    df.rename(columns={'YearMonth_x': 'YearMonth'}, inplace=True)
    globals()[f'product{i}'] = df

In [200]:
# Make DATE the index of the DataFrame
for i in range(1, 15):
    df = globals()[f'product{i}']
    df = df.set_index('DATE')
    globals()[f'product{i}'] = df

In [201]:
product1.columns

Index(['YearMonth', 'Mapped_GCK', 'Sales_EUR', 'China_MAB_ELE_PRO156',
       'China_MAB_ELE_SHP156', 'France_MAB_ELE_PRO250',
       'France_MAB_ELE_SHP250', 'Germany_MAB_ELE_PRO276',
       'Germany_MAB_ELE_SHP276', 'Italy_MAB_ELE_PRO380',
       'Italy_MAB_ELE_SHP380', 'Japan_MAB_ELE_PRO392', 'Japan_MAB_ELE_SHP392',
       'Switzerland_MAB_ELE_PRO756', 'Switzerland_MAB_ELE_SHP756',
       'United Kingdom_MAB_ELE_PRO826', 'United Kingdom_MAB_ELE_SHP826',
       'United States_MAB_ELE_PRO840', 'United States_MAB_ELE_SHP840',
       'Europe_MAB_ELE_PRO1100', 'Europe_MAB_ELE_SHP1100',
       'Europe_RohiBASEMET1000_org', 'Europe_RohiENERGY1000_org',
       'Europe_RohiMETMIN1000_org', 'Europe_RohiNATGAS1000_org',
       'Europe_RohCRUDE_PETRO1000_org', 'Europe_RohCOPPER1000_org',
       'Europe_WKLWEUR840_org', 'Producer Prices_PRI27840_org',
       'Producer Prices_PRI27826_org', 'Producer Prices_PRI27380_org',
       'Producer Prices_PRI27250_org', 'Producer Prices_PRI27276_org',
    

In [202]:
product4.shape

(43, 50)

In [203]:
# check if the change was successful
for i in range(1, len(product_dataframes) + 1):
     print(f"product{i} shape is:{globals()[f'product{i}'].shape}")

product1 shape is:(43, 50)
product2 shape is:(43, 50)
product3 shape is:(43, 50)
product4 shape is:(43, 50)
product5 shape is:(43, 50)
product6 shape is:(43, 50)
product7 shape is:(43, 50)
product8 shape is:(43, 50)
product9 shape is:(43, 50)
product10 shape is:(43, 50)
product11 shape is:(43, 50)
product12 shape is:(43, 50)
product13 shape is:(43, 50)
product14 shape is:(43, 50)


In [204]:
# check if the change was successful
for i in range(1, len(product_dataframes) + 1):
     print(f"product{i} first 5 rows:{globals()[f'product{i}'].head()}")

product1 first 5 rows:           YearMonth Mapped_GCK    Sales_EUR  China_MAB_ELE_PRO156  \
DATE                                                                 
2018-10-01   2018-10   Product1  36098918.79            211.955755   
2018-11-01   2018-11   Product1   5140760.00            220.519655   
2018-12-01   2018-12   Product1  37889612.12            241.846854   
2019-01-01   2019-01   Product1  27728148.35            175.668147   
2019-02-01   2019-02   Product1  34793163.53            175.668147   

            China_MAB_ELE_SHP156  France_MAB_ELE_PRO250  \
DATE                                                      
2018-10-01            211.955755             108.280608   
2018-11-01            220.519655              99.636911   
2018-12-01            241.846854              94.690312   
2019-01-01            175.668147              90.143775   
2019-02-01            175.668147              92.551521   

            France_MAB_ELE_SHP250  Germany_MAB_ELE_PRO276  \
DATE        

# Train-test split

# Feature Engineering

## Lag features for Sales_EUR

`Lag features` answer the question "what were the sales X months ago?" - we are using lag with X=[1,3,12] (1 month, 1 trimester, 1 year)

In [206]:
# include recent sales for each product
for i in range(1, 15):
    df = globals()[f'product{i}']
    df['Sales_Lag_1'] = df['Sales_EUR'].shift(1)
    df['Sales_Lag_3'] = df['Sales_EUR'].shift(3)
    df['Sales_Lag_12'] = df['Sales_EUR'].shift(12)
    globals()[f'product{i}'] = df

`Rolling mean(moving average)`capture short-term trends or smooths out noise. For example, April 2021, it averages Jan–Mar 2021 (but shifted 1 step so we're not using the current month’s value).


In [207]:
## Include rolling mean for each product
for i in range(1, 15):
    df = globals()[f'product{i}']
    df['Rolling_Mean_3'] = df['Sales_EUR'].shift(1).rolling(window=3).mean() # mean of the last 3 months
    df['Rolling_Mean_6'] = df['Sales_EUR'].shift(1).rolling(window=6).mean() # mean of the last 6 months
    globals()[f'product{i}'] = df

## Temporal features

We are creating columns to keep the month, and the quarter, as it may help identify seasonal trends

In [217]:
# create a month and a quarter column
for i in range(1, 15):
    df = globals()[f'product{i}']
    df['Month'] = df.index.month
    df['Quarter'] = df.index.quarter
    globals()[f'product{i}'] = df

## Lag features for Macro Variables

In [218]:
# Loop over all 14 product DataFrames
for i in range(1, 15):
    df = globals()[f'product{i}']
    
    # Identify macro columns (exclude sales/date/product ID columns)
    macro_cols = [col for col in df.columns if col not in [
        'YearMonth_x', 'YearMonth_y', 'Mapped_GCK', 'Sales_EUR', 'DATE'
    ] and not col.startswith('Sales') and not col in ['Month', 'Quarter', 'Product_ID']]
    
    # Create lag and delta features
    for col in macro_cols:
        df[f'{col}_Lag1'] = df[col].shift(1)
        df[f'{col}_Delta'] = df[col] - df[col].shift(1)
    
    # Save back to global scope
    globals()[f'product{i}'] = df

# Feature selection

# train/test split

# Model with XGBoost (ML Approach)

In [209]:
# results = []

# # Loop through each product
# for gck in full_df['Mapped_GCK'].unique():
#     gck_df = full_df[full_df['Mapped_GCK'] == gck].dropna()
    
#     train = gck_df[gck_df['YearMonth'] < '2022-05-01']
#     test = gck_df[gck_df['YearMonth'] >= '2022-05-01']
    
#     features = [col for col in gck_df.columns if col not in ['Sales_EUR', 'Date', 'YearMonth', 'Mapped_GCK']]
    
#     model = XGBRegressor(n_estimators=100, learning_rate=0.1)
#     model.fit(train[features], train['Sales_EUR'])
    
#     preds = model.predict(test[features])
    
#     rmse = np.sqrt(mean_squared_error(test['Sales_EUR'], preds))
#     results.append((gck, rmse))
    
#     # Save predictions for submission
#     test['Predicted_Sales'] = preds
#     test[['YearMonth', 'Mapped_GCK', 'Predicted_Sales']].to_csv(f'predictions_{gck}.csv', index=False)

# Model with SARIMAX (Statistical Approach)

In [210]:
# # Example for 1 product
# gck = 'Product_X'
# gck_df = full_df[full_df['Mapped_GCK'] == gck].set_index('YearMonth')

# endog = gck_df['Sales_EUR']
# exog = gck_df[market_df.columns.difference(['Date', 'YearMonth'])]

# # Train/test split
# train_endog = endog[:'2022-04']
# test_endog = endog['2022-05':]

# train_exog = exog.loc[:'2022-04']
# test_exog = exog.loc['2022-05':]

# model = SARIMAX(train_endog, exog=train_exog, order=(1,1,1), seasonal_order=(1,1,1,12))
# sarimax_res = model.fit()

# forecast = sarimax_res.predict(start=test_endog.index[0], end=test_endog.index[-1], exog=test_exog)
# rmse = np.sqrt(mean_squared_error(test_endog, forecast))
# print(f"RMSE for {gck}: {rmse}")

# Evaluation

# Submission Formatting

In [211]:
# # Combine all test predictions into one file
# final_submission = pd.concat([pd.read_csv(f'predictions_{gck}.csv') for gck in full_df['Mapped_GCK'].unique()])
# final_submission.columns = ['Year Month', 'Mapped_GCK', 'Sales EUR']
# final_submission.to_csv("Submission_Template.csv", index=False)